In [2]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

2022-07-25 14:02:23.590803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-25 14:02:23.590838: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test

In [6]:
# Single Point on Sphere

n = 40000
rand_n = 12345
theta, phi = 3, 3
thetas, phis = theta * np.ones(n), phi * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

model = tf.keras.models.load_model('3dmodels/discrete_model_mth15_mph15')

In [7]:
for l in model.layers:
    l.trainable=False

inputs_hold = tf.keras.Input(shape=(1,))
simple_model = Dense(2, use_bias = False)(inputs_hold)
model3 = Model(inputs = inputs_hold, outputs = simple_model)

inputs = tf.keras.Input(shape=(2,))
inputs2 = tf.keras.layers.concatenate([inputs, model3(tf.ones_like(inputs)[:,0:1])])
hidden_layer_1 = model(inputs2)

model2 = Model(inputs = inputs, outputs = hidden_layer_1)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model2.compile(loss=loss_fn, optimizer='Adam', )

model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like (TFOpLambda)       (None, 2)            0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1)            0           tf.ones_like[0][0]               
__________________________________________________________________________________________________
model (Functional)              (None, 2)            2           tf.__operators__.getitem[0][0]   
____________________________________________________________________________________________

In [8]:
coordinates = x_train[:, 0:2]

for i in range(10):
    model2.fit(coordinates, y_train, epochs=1,batch_size=100)
    print(i,"Fitted result: ", model2.trainable_weights[:][0][0])
    x = model2(coordinates).numpy().transpose()[0]
    x_r = real_model(x_train).numpy().transpose()[0]
    print(np.sum(-y_train * np.log(x) - (1-y_train) * np.log(1-x)) / x.shape[0])
    print(np.sum(-y_train * np.log(x_r) - (1-y_train) * np.log(1-x_r)) / x.shape[0])

2022-07-25 14:06:15.137879: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


640/640 [==============================] - 2s 2ms/step - loss: 0.7114
0 Fitted result:  tf.Tensor([ 1.3701652 -0.6324784], shape=(2,), dtype=float32)
0.6818731711196014
640/640 [==============================] - 1s 2ms/step - loss: 0.6729
1 Fitted result:  tf.Tensor([ 1.1928481  -0.23789924], shape=(2,), dtype=float32)
0.6584915357102582
640/640 [==============================] - 1s 2ms/step - loss: 0.6485
2 Fitted result:  tf.Tensor([0.9548949  0.18855423], shape=(2,), dtype=float32)
0.6338355576706963
640/640 [==============================] - 1s 2ms/step - loss: 0.6256
3 Fitted result:  tf.Tensor([0.80778867 0.56594545], shape=(2,), dtype=float32)
0.6163021088415263
640/640 [==============================] - 1s 2ms/step - loss: 0.6148
4 Fitted result:  tf.Tensor([0.7700897 0.8058613], shape=(2,), dtype=float32)
0.6105814371670131
640/640 [==============================] - 1s 2ms/step - loss: 0.6117
5 Fitted result:  tf.Tensor([0.84381217 0.98893845], shape=(2,), dtype=float32)
0.608

In [10]:
real_model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')

In [13]:
x_r = real_model(x_train).numpy().transpose()[0]

In [17]:
np.sum(-y_train * np.log(x_r) - (1-y_train) * np.log(1-x_r)) / x_r.shape[0]

0.5571606174300833